# Testes de Hipótese Unilaterais
Aqui exploraremos a aplicação de testes de hipótese direcionados (unilaterais), definindo a região crítica em apenas uma cauda da distribuição.

In [2]:
import numpy as np
from scipy import stats

## Parâmetros de controle do experimento - Altere esses valores para testar novos cenários
x1    =  15000  # Média de erro modelo A
x2    =  14500  # Média de erro modelo B
s1    =  5000   # Desvio padrão de erro modelo A
s2    =  4800   # Desvio padrão de erro modelo B
n1    =  500    # Tamanho da amostra modelo A
n2    =  500    # Tamanho da amostra modelo B
alpha =  0.10   # Nível de significância

# Define uma semente aleatória para garantir que os resultados sejam sempre os mesmos.
np.random.seed(28)

# Simulação dos erros de cada modelo para nA e nB previsões através de dados sintéticos aleatórios
# Modelo Antigo (A): Erro médio de xA com desvio padrão de sA
erros_modelo_A = np.random.normal(loc=x1, scale=s1, size=n1)

# Novo Modelo (B): Erro médio de xB com desvio padrão de sB
erros_modelo_B = np.random.normal(loc=x2, scale=s2, size=n2)

# Vamos verificar as médias observadas em nossa simulação
media_erros_A = np.mean(erros_modelo_A)
media_erros_B = np.mean(erros_modelo_B)

print(f"Média de Erro do Modelo A: R$ {media_erros_A:,.2f}")
print(f"Média de Erro do Modelo B: R$ {media_erros_B:,.2f}")
print(f"Diferença observada: R$ {media_erros_A - media_erros_B:,.2f}")

Média de Erro do Modelo A: R$ 14,879.74
Média de Erro do Modelo B: R$ 14,478.44
Diferença observada: R$ 401.31


In [3]:
# H₀: as médias são iguais
# H₁: a média de A é MAIOR que a de B (ou a de B é MENOR que a de A)
t_stat, p_valor = stats.ttest_ind(erros_modelo_A, erros_modelo_B, alternative='greater')

print(f"Valor-p calculado: {p_valor:.6f}")

if p_valor <= alpha:
    print("Conclusão: Rejeitamos a H₀. O novo modelo tem uma redução de erro estatisticamente significativa.")
else:
    print("Conclusão: Falhamos em rejeitar a H₀. Não há evidência suficiente de que o novo modelo é melhor.")

Valor-p calculado: 0.094223
Conclusão: Rejeitamos a H₀. O novo modelo tem uma redução de erro estatisticamente significativa.


In [4]:
# Calculando o Intervalo de Confiança para a diferença das médias
diferenca_medias = media_erros_A - media_erros_B
nA, nB = len(erros_modelo_A), len(erros_modelo_B)
sA, sB = np.std(erros_modelo_A, ddof=1), np.std(erros_modelo_B, ddof=1)

# Graus de liberdade
gl = nA + nB - 2

# Erro padrão da diferença
erro_padrao_diff = np.sqrt((sA**2 / nA) + (sB**2 / nB))

# A probabilidade acumulada (ou percentile) para o nível de significância estabelecido
q = 1 - alpha / 2

# t-crítico para o q equivalente ao nível de significância estabelecido
t_critico = stats.t.ppf(q, df=gl)

# Margem de erro
margem_erro = t_critico * erro_padrao_diff

# Limites do intervalo
limite_inferior = diferenca_medias - margem_erro
limite_superior = diferenca_medias + margem_erro

print(f"A melhoria estimada é de R$ {diferenca_medias:,.2f}.")
print(f"O Intervalo de Confiança de {(1-alpha)*100:,.0f}% para essa melhoria é: [R$ {limite_inferior:,.2f}, R$ {limite_superior:,.2f}]")

A melhoria estimada é de R$ 401.31.
O Intervalo de Confiança de 90% para essa melhoria é: [R$ -100.71, R$ 903.33]


In [5]:
from typing import Literal
import numpy as np
from scipy import stats

def calcular_limite_confianca_unilateral(
    amostra_a: np.ndarray,
    amostra_b: np.ndarray,
    tipo: Literal['LCL', 'UCL'],
    alpha: float = 0.05,
    ddof: int = 1
) -> tuple[float, float]:
    """
    Calcula um limite de confiança unilateral para a diferença entre as médias de duas amostras independentes.

    Esta função é ideal para determinar o limite inferior (LCL) ou superior (UCL) 
    da diferença entre duas médias, correspondendo a um teste de hipótese unilateral.

    Args:
        amostra_a (np.ndarray): Array de NumPy com as observações do primeiro grupo (grupo de controle).
        amostra_b (np.ndarray): Array de NumPy com as observações do segundo grupo (grupo de teste).
        tipo (Literal['LCL', 'UCL']): O tipo de limite a ser calculado. Este parâmetro é obrigatório.
                                     'LCL' para Limite de Confiança Inferior (Lower Confidence Limit).
                                     'UCL' para Limite de Confiança Superior (Upper Confidence Limit).
        alpha (float, optional): Nível de significância para o teste. O nível de confiança será (1 - alpha). 
                                 Padrão é 0.05 (confiança de 95%).
        ddof (int, optional): Delta Degrees of Freedom. O divisor usado no cálculo do desvio padrão é N - ddof.
                              O padrão é 1 (Correção de Bessel), que fornece uma estimativa não enviesada 
                              do desvio padrão da população a partir de uma amostra. Use 0 se seus dados
                              representarem a população inteira.

    Returns:
        tuple[float, float]: Uma tupla contendo o limite inferior e o limite superior calculados.
    """
    # Passo 1: Calcular a diferença observada entre as médias das amostras. Este é o nosso "Sinal".
    diferenca_medias = np.mean(amostra_a) - np.mean(amostra_b)
    
    # Passo 2: Calcular as estatísticas de cada amostra (tamanho e desvio padrão).
    # O parâmetro 'ddof' ajusta o cálculo do desvio padrão. O padrão ddof=1 é para amostras e o ddof=0 para populção.
    nA, nB = len(amostra_a), len(amostra_b)
    sA, sB = np.std(amostra_a, ddof=ddof), np.std(amostra_b, ddof=ddof)
    
    # Passo 3: Calcular os graus de liberdade (degrees of freedom) para a Distribuição T.
    gl = nA + nB - 2
    
    # Passo 4: Calcular o "Ruído" - o Erro Padrão da Diferença entre as médias.
    erro_padrao_diff = np.sqrt((sA**2 / nA) + (sB**2 / nB))
    
    # Passo 5: Encontrar o t_crítico a partir da Distribuição t.
    q = 1 - alpha
    t_critico = stats.t.ppf(q, df=gl)
    
    # Passo 6: Calcular a Margem de Erro.
    margem_erro = t_critico * erro_padrao_diff
    
    # Passo 7: Calcular o limite final (piso ou teto) com base no tipo especificado.
    if tipo == 'LCL':
        limite_inferior = diferenca_medias - margem_erro
        limite_superior = np.inf
    elif tipo == 'UCL':
        limite_inferior = -np.inf
        limite_superior = diferenca_medias + margem_erro
    else:
        raise ValueError(f"Parâmetro 'tipo' inválido. Deve ser 'LCL' ou 'UCL', mas foi '{tipo}'.")
        
    return limite_inferior, limite_superior

if __name__ == '__main__':
    # Limites do intervalo
    limite_inferior, limite_superior = calcular_limite_confianca_unilateral(erros_modelo_A, erros_modelo_B,
                                                                            alpha=alpha, tipo='LCL')
    
    print(f"O Intervalo Unilateral de {(1-alpha)*100:,.0f}% é: [{limite_inferior:,.2f}, {limite_superior:,.2f}]")

O Intervalo Unilateral de 90% é: [10.27, inf]
